In [ ]:
from __future__ import print_function

"""
A script to run cqed-rhf and cqed-cis method on formaldehyde coupled to a photon approximately 
resonant with the first 2 optically allowed transitions polarized along $z$ or $yz$ to show how the cqed-cis ground state is
stabilized relative to the cqed-rhf solution, reproducing Table II and Figure 2
data by McTague and Foley.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from helper_cis import *
from matplotlib import pyplot as plt
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [ ]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


rhf_energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
print(rhf_energy)
    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=5, triplets = "NONE", tda=True)


# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

psi4_tdm = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]




# run helper_cis()
#cis_e, cis_c, cis_mu, tdm = cis(molstr, options_dict)

print("psi4 dE")
print(psi4_excitation_e[0] * 27.211, psi4_excitation_e[1] * 27.211, psi4_excitation_e[2] * 27.211, psi4_excitation_e[3] * 27.211, psi4_excitation_e[4] * 27.211)
print("psi4 transition dipole moments")
print(psi4_tdm)

Compute CQED-CIS ground state for the $\lambda_z$ and $\lambda_{yz}$ cases.

In [ ]:
# magnitude of coupling
l = 0.2

# photon frequency
om = 10.4/27.211

no_lam = np.array([0., 0., 0.])
lam_z = np.array([0, 0, l])
lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])
    
# cqed-rhf results 
yz_cqed_rhf_dict = cqed_rhf(lam_yz,molstr, options_dict)
z_cqed_rhf_dict = cqed_rhf(lam_z, molstr, options_dict)

# cqed-cis results 
yz_cqed_cis_dict = cs_cqed_cis(lam_yz, om, molstr, options_dict)
z_cqed_cis_dict = cs_cqed_cis(lam_z, om, molstr, options_dict)


print("cqed_cis energy relative to rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0]-rhf_energy)
print("cqed_cis energy relative to rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0]-rhf_energy)




In [ ]:
print("cqed_cis energy relative to rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0]+z_cqed_rhf_dict['CQED-RHF ENERGY']-rhf_energy)
print("cqed_cis energy relative to rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0]+yz_cqed_rhf_dict['CQED-RHF ENERGY']-rhf_energy)


print("cqed_cis energy relative to cqed-rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0])
print("cqed_cis energy relative to cqed-rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0])


Now analyze the transitions that contribute to the CQED-CIS ground state
for the $\lambda_{yz}$ 

In [ ]:

# get orbital energies for both of these polarizations
eps_yz = yz_cqed_rhf_dict["CQED-RHF EPS"]
eps_eV = 27.211 * eps_yz


# number of doubly occupied orbitals
ndocc   = wfn.nalpha()
    
# total number of orbitals
nmo     = wfn.nmo()

nvirt = nmo - ndocc
    
# Note yz-polarized field belongs to A' 
cs_irrep = ["1A'", "2A'", "3A'", "4A'", "5A'", "6A'", "1A''", "7A'", "2A''", "8A'", "9A'", "10A'", "3A''", 
            "11A'", "12A'"]


lam_yz_c = yz_cqed_cis_dict['CQED-CIS L VECTORS']
state=0
print(" excitation with lambda_yz")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_yz_c[ia0,state] * np.conj(lam_yz_c[ia0,state])
        amp01 = lam_yz_c[ia1,state] * np.conj(lam_yz_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00," : ", cs_irrep[i], "->",cs_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01," : ", cs_irrep[i], "->",cs_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
            


Now analyze the transitions that contribute to the CQED-CIS ground state
for the $\lambda_{z}$ 

In [ ]:
# results for z-polarized
# get orbital energies for both of these polarizations
eps_z = z_cqed_rhf_dict["CQED-RHF EPS"]
eps_eV = 27.211 * eps_z


# number of doubly occupied orbitals
ndocc   = wfn.nalpha()
    
# total number of orbitals
nmo     = wfn.nmo()

nvirt = nmo - ndocc


    
# Note y-polarized field belongs to B2 and z-polarized field belongs to A1
c2v_irrep = ["1A1", "2A1", "3A1", "4A1", "1B2", "5A1", "1B1", "2B2", 
         "2B1", "6A1", "3B2", "7A1","3B1", "8A1", "4B2", "5B2", "9A1"]

lam_z_c = z_cqed_cis_dict['CQED-CIS L VECTORS']
state=0
print(" excitation with lambda_z")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_z_c[ia0,state] * np.conj(lam_z_c[ia0,state])
        amp01 = lam_z_c[ia1,state] * np.conj(lam_z_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00," : ", c2v_irrep[i], "->",c2v_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01," : ", c2v_irrep[i], "->",c2v_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
            